# Data cleaning 101

In [129]:
import argparse
from fileinput import filename
import os
from pathlib import Path
import pandas as pd
import numpy as np
from dateutil.parser import parse
from tabnanny import verbose

In [152]:
data_source = Path(".").resolve().parent
listmonth = ["enero","febrero","marzo","abril","mayo","junio","julio","agosto","septiembre","octubre","noviembre","diciembre"]
listyear = ["2021","2022"]
mes = str(input("Por favor ingrese el mes que quiere analizar: "))
ano = str(input("Por favor ingrese el año que quiere analizar: "))
mes = mes.strip()
ano = ano.strip()
try: 
    if mes not in listmonth and ano not in listyear:
        raise ValueError("No se encuentra este mes y año aún disponible para analizar. Por favor intente otra combinación")
    else:
        file_name = "llamadas123_"+mes+"_"+ano+".csv"
        print(file_name)
except ValueError as ve:
    print(ve) 

llamadas123_diciembre_2021.csv


In [153]:
data_raw = os.path.join(data_source,"data","raw",file_name)
datos = pd.read_csv(data_raw, sep =";", encoding="latin-1")
try: 
    if datos.shape[1] != 11:
        raise ValueError("Revise el # de columnas del archivo antes de ingresarlo para analizar")
    else:
        datos.columns = datos.columns.str.replace('[#,@,&,-]', '_')
        datos.columns = ["NUMERO_INCIDENTE","FECHA_INICIO_DESPLAZAMIENTO_MOVIL","CODIGO_LOCALIDAD","LOCALIDAD","EDAD","UNIDAD","GENERO","RED","TIPO_INCIDENTE","PRIORIDAD","RECEPCION"]
except ValueError as ve:
    print(ve)

C:\Users\57320\AppData\Local\Temp\ipykernel_9768\303913390.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  datos.columns = datos.columns.str.replace('[#,@,&,-]', '_')


1. #Obtener informacion basica de los datos

In [154]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13771 entries, 0 to 13770
Data columns (total 11 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   NUMERO_INCIDENTE                   13771 non-null  object
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  13771 non-null  object
 2   CODIGO_LOCALIDAD                   13771 non-null  int64 
 3   LOCALIDAD                          13771 non-null  object
 4   EDAD                               13771 non-null  object
 5   UNIDAD                             13770 non-null  object
 6   GENERO                             13771 non-null  object
 7   RED                                13771 non-null  object
 8   TIPO_INCIDENTE                     13771 non-null  object
 9   PRIORIDAD                          13771 non-null  object
 10  RECEPCION                          13771 non-null  object
dtypes: int64(1), object(10)
memory usage: 1.2+ MB


1. # Eliminar registros duplicados

In [155]:
print("forma inicial", datos.shape)
df_no_dupl = datos.drop_duplicates() #drop_duplicates es para eliminar duplicados de una tabla
print("forma final", df_no_dupl.shape)
#vemos con esto la diferencia
df_no_dupl.reset_index() #aqui se ve que se crea una columna index


forma inicial (13771, 11)
forma final (13754, 11)


,index,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,RECEPCION
0,0,CRU-00595688-21,1/12/2021 0:25,18,Rafael Uribe Uribe,67,A¤os,MASCULINO,Sur,Enfermo,MEDIA,1/12/2021 3:40
1,1,CRU-00595689-21,1/12/2021 0:09,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Herido,BAJA,1/12/2021 3:38
2,2,CRU-00595690-21,1/12/2021 0:24,4,San Crist¢bal,34,A¤os,MASCULINO,Centro Oriente,Enfermo,MEDIA,1/12/2021 4:12
3,3,CRU-00595693-21,1/12/2021 0:13,7,Bosa,SIN_DATO,SIN_DATO,SIN_DATO,Sur Occidente,Trastorno Mental,CRITICA,0000-00-00 00:00:00
4,4,CRU-00595694-21,1/12/2021 0:14,3,Santa Fe,61,A¤os,FEMENINO,Centro Oriente,Accidente Cerebro Vascular,CRITICA,1/12/2021 3:25
...,...,...,...,...,...,...,...,...,...,...,...,...
13749,13766,CRU-00646044-21,31/12/2021 23:55,3,Santa Fe,SIN_DATO,SIN_DATO,SIN_DATO,Centro Oriente,Herido,CRITICA,0000-00-00 00:00:00
13750,13767,CRU-00646046-21,31/12/2021 23:53,19,Ciudad Bol¡var,SIN_DATO,SIN_DATO,SIN_DATO,Sur,Herido,ALTA,0000-00-00 00:00:00
13751,13768,CRU-00646048-21,1/01/2022 0:11,15,Antonio Nari¤o,30,A¤os,FEMENINO,Centro Oriente,Intoxicaci¢n,ALTA,0000-00-00 00:00:00
13752,13769,CRU-00646050-21,31/12/2021 23:54,19,Ciudad Bol¡var,16,A¤os,MASCULINO,Sur,Evento Respiratorio,CRITICA,30/12/2021 23:34


In [156]:
#hallamos los datos duplicados que se borraron en el codigo anterior
datos[
    datos.index.isin(df_no_dupl.index) == False
]

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,RECEPCION
166,CRU-00595955-21,1/12/2021 8:32,16,Puente Aranda,21,A¤os,MASCULINO,Sur Occidente,Herido,MEDIA,0000-00-00 00:00:00
295,CRU-00596418-21,1/12/2021 14:37,16,Puente Aranda,32,A¤os,FEMENINO,Norte,Intoxicaci¢n,ALTA,0000-00-00 00:00:00
1135,CRU-00599201-21,3/12/2021 13:58,2,Chapinero,SIN_DATO,SIN_DATO,SIN_DATO,Sur Occidente,Evento Respiratorio,CRITICA,0000-00-00 00:00:00
3264,CRU-00612712-21,8/12/2021 2:49,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Herido,ALTA,0000-00-00 00:00:00
3298,CRU-00612940-21,8/12/2021 4:14,8,Kennedy,SIN_DATO,SIN_DATO,SIN_DATO,Sur Occidente,Inconsciente / Paro cardiorespiratorio,CRITICA,0000-00-00 00:00:00
5255,CRU-00614512-21,12/12/2021 9:20,1,Usaqun,53,A¤os,FEMENINO,Norte,Inconsciente / Paro cardiorespiratorio,CRITICA,0000-00-00 00:00:00
5763,CRU-00616215-21,13/12/2021 11:44,18,Rafael Uribe Uribe,27,A¤os,FEMENINO,Centro Oriente,Convulsi¢n,ALTA,0000-00-00 00:00:00
5782,CRU-00616296-21,13/12/2021 13:01,19,Ciudad Bol¡var,SIN_DATO,SIN_DATO,SIN_DATO,Sur,Inconsciente / Paro cardiorespiratorio,CRITICA,0000-00-00 00:00:00
7746,CRU-00623060-21,18/12/2021 1:13,12,Barrios Unidos,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Herido,ALTA,0000-00-00 00:00:00
8200,CRU-00624654-21,18/12/2021 23:23,9,Fontib¢n,SIN_DATO,SIN_DATO,SIN_DATO,Sur Occidente,Herido,BAJA,0000-00-00 00:00:00


In [157]:
# hay que resetear el indice para que este organizada la tabla y sin saltos después de  hacer los duplicados
df_no_dupl.reset_index(inplace=True, drop=True) #con estos parametros borramos la columna index que nos genera 
print("forma final", df_no_dupl.shape)
df_no_dupl


forma final (13754, 11)


,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,RECEPCION
0,CRU-00595688-21,1/12/2021 0:25,18,Rafael Uribe Uribe,67,A¤os,MASCULINO,Sur,Enfermo,MEDIA,1/12/2021 3:40
1,CRU-00595689-21,1/12/2021 0:09,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Herido,BAJA,1/12/2021 3:38
2,CRU-00595690-21,1/12/2021 0:24,4,San Crist¢bal,34,A¤os,MASCULINO,Centro Oriente,Enfermo,MEDIA,1/12/2021 4:12
3,CRU-00595693-21,1/12/2021 0:13,7,Bosa,SIN_DATO,SIN_DATO,SIN_DATO,Sur Occidente,Trastorno Mental,CRITICA,0000-00-00 00:00:00
4,CRU-00595694-21,1/12/2021 0:14,3,Santa Fe,61,A¤os,FEMENINO,Centro Oriente,Accidente Cerebro Vascular,CRITICA,1/12/2021 3:25
...,...,...,...,...,...,...,...,...,...,...,...
13749,CRU-00646044-21,31/12/2021 23:55,3,Santa Fe,SIN_DATO,SIN_DATO,SIN_DATO,Centro Oriente,Herido,CRITICA,0000-00-00 00:00:00
13750,CRU-00646046-21,31/12/2021 23:53,19,Ciudad Bol¡var,SIN_DATO,SIN_DATO,SIN_DATO,Sur,Herido,ALTA,0000-00-00 00:00:00
13751,CRU-00646048-21,1/01/2022 0:11,15,Antonio Nari¤o,30,A¤os,FEMENINO,Centro Oriente,Intoxicaci¢n,ALTA,0000-00-00 00:00:00
13752,CRU-00646050-21,31/12/2021 23:54,19,Ciudad Bol¡var,16,A¤os,MASCULINO,Sur,Evento Respiratorio,CRITICA,30/12/2021 23:34


In [158]:
# Tratamiento de valores nulos
# nulo de strings --> SIN_DATO, N/A
# nulo de numeros --> np.nan
# nulo para fechas --> NaT

col = "UNIDAD"
df_no_dupl[col].fillna("SIN_DATO", inplace=True) #llenamos los nan con el string "SIN_DATO" y reemplazamos en la tabla
df_no_dupl[col].value_counts(dropna = False, normalize=True) #con dropna si es Falso tenemos en cuenta los NaN y si es True no los vemos


C:\Users\57320\AppData\Local\Temp\ipykernel_9768\4025764682.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_dupl[col].fillna("SIN_DATO", inplace=True) #llenamos los nan con el string "SIN_DATO" y reemplazamos en la tabla


A¤os        0.516432
SIN_DATO    0.479788
Meses       0.002472
Horas       0.000727
Dias        0.000582
Name: UNIDAD, dtype: float64

In [159]:
df_no_dupl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13754 entries, 0 to 13753
Data columns (total 11 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   NUMERO_INCIDENTE                   13754 non-null  object
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  13754 non-null  object
 2   CODIGO_LOCALIDAD                   13754 non-null  int64 
 3   LOCALIDAD                          13754 non-null  object
 4   EDAD                               13754 non-null  object
 5   UNIDAD                             13754 non-null  object
 6   GENERO                             13754 non-null  object
 7   RED                                13754 non-null  object
 8   TIPO_INCIDENTE                     13754 non-null  object
 9   PRIORIDAD                          13754 non-null  object
 10  RECEPCION                          13754 non-null  object
dtypes: int64(1), object(10)
memory usage: 1.2+ MB


In [160]:
col_date = "FECHA_INICIO_DESPLAZAMIENTO_MOVIL"
df_no_dupl[col_date] = pd.to_datetime(df_no_dupl[col_date], errors ="coerce",format="%Y/%m/%d")
df_no_dupl.info()
#vamos a empezar a estandarizar la fecha #coerse es para que si me aparecen datos con errores me aparezcan como NaT

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13754 entries, 0 to 13753
Data columns (total 11 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   NUMERO_INCIDENTE                   13754 non-null  object        
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  0 non-null      datetime64[ns]
 2   CODIGO_LOCALIDAD                   13754 non-null  int64         
 3   LOCALIDAD                          13754 non-null  object        
 4   EDAD                               13754 non-null  object        
 5   UNIDAD                             13754 non-null  object        
 6   GENERO                             13754 non-null  object        
 7   RED                                13754 non-null  object        
 8   TIPO_INCIDENTE                     13754 non-null  object        
 9   PRIORIDAD                          13754 non-null  object        
 10  RECEPCION                         

C:\Users\57320\AppData\Local\Temp\ipykernel_9768\3590230399.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_dupl[col_date] = pd.to_datetime(df_no_dupl[col_date], errors ="coerce",format="%Y/%m/%d")


In [161]:
col_edad = "EDAD"
df_no_dupl[col_edad].replace({"SIN_DATO": np.nan, "Sin_dato": np.nan}, inplace =True) #reemplazo SIN_DATO por valor nan en la columna EDAD
#df_no_dupl[col_edad].replace({"Sin_dato": np.nan}, inplace =True) #reemplazo Sin_dato por valor nan en la columna EDAD
df_no_dupl[col_edad].value_counts("SIN_DATOS")

C:\Users\57320\AppData\Local\Temp\ipykernel_9768\1576276864.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_dupl[col_edad].replace({"SIN_DATO": np.nan, "Sin_dato": np.nan}, inplace =True) #reemplazo SIN_DATO por valor nan en la columna EDAD


25    0.025852
22    0.024455
21    0.022499
26    0.021101
23    0.020542
        ...   
95    0.001956
97    0.001677
7     0.001397
96    0.001397
98    0.001118
Name: EDAD, Length: 99, dtype: float64

In [114]:
df_no_dupl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13754 entries, 0 to 13753
Data columns (total 11 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   NUMERO_INCIDENTE                   13754 non-null  object        
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  0 non-null      datetime64[ns]
 2   CODIGO_LOCALIDAD                   13754 non-null  int64         
 3   CODIGO_LOCALIDAD                   13754 non-null  object        
 4   EDAD                               13754 non-null  object        
 5   UNIDAD                             13753 non-null  object        
 6   GENERO                             13754 non-null  object        
 7   RED                                13754 non-null  object        
 8   TIPO_INCIDENTE                     13754 non-null  object        
 9   PRIORIDAD                          13754 non-null  object        
 10  RECEPCION                         

In [163]:
#funciones lambda
# def f(x):
#    resultado = x**2
#    return resultado
string_num = "28"
f = lambda x: x if pd.isna(x) else int(x)
f(string_num)

28

In [162]:
df_no_dupl[df_no_dupl[col_edad].isin(["SIN_DATO"])] #filtramos por los valores Sin_dato para verificar que no haya ninguno y se hallan reemplazado todos

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,RECEPCION


In [164]:
df_no_dupl[col_edad] = df_no_dupl[col_edad].apply(f) #aplicamos la funcion f a esta columna

C:\Users\57320\AppData\Local\Temp\ipykernel_9768\377609343.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_dupl[col_edad] = df_no_dupl[col_edad].apply(f) #aplicamos la funcion f a esta columna


In [165]:
df_no_dupl.info()
df_no_dupl

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13754 entries, 0 to 13753
Data columns (total 11 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   NUMERO_INCIDENTE                   13754 non-null  object        
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  0 non-null      datetime64[ns]
 2   CODIGO_LOCALIDAD                   13754 non-null  int64         
 3   LOCALIDAD                          13754 non-null  object        
 4   EDAD                               7156 non-null   float64       
 5   UNIDAD                             13754 non-null  object        
 6   GENERO                             13754 non-null  object        
 7   RED                                13754 non-null  object        
 8   TIPO_INCIDENTE                     13754 non-null  object        
 9   PRIORIDAD                          13754 non-null  object        
 10  RECEPCION                         

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,RECEPCION
0,CRU-00595688-21,NaT,18,Rafael Uribe Uribe,67.0,A¤os,MASCULINO,Sur,Enfermo,MEDIA,1/12/2021 3:40
1,CRU-00595689-21,NaT,11,Suba,NaN,SIN_DATO,SIN_DATO,Norte,Herido,BAJA,1/12/2021 3:38
2,CRU-00595690-21,NaT,4,San Crist¢bal,34.0,A¤os,MASCULINO,Centro Oriente,Enfermo,MEDIA,1/12/2021 4:12
3,CRU-00595693-21,NaT,7,Bosa,NaN,SIN_DATO,SIN_DATO,Sur Occidente,Trastorno Mental,CRITICA,0000-00-00 00:00:00
4,CRU-00595694-21,NaT,3,Santa Fe,61.0,A¤os,FEMENINO,Centro Oriente,Accidente Cerebro Vascular,CRITICA,1/12/2021 3:25
...,...,...,...,...,...,...,...,...,...,...,...
13749,CRU-00646044-21,NaT,3,Santa Fe,NaN,SIN_DATO,SIN_DATO,Centro Oriente,Herido,CRITICA,0000-00-00 00:00:00
13750,CRU-00646046-21,NaT,19,Ciudad Bol¡var,NaN,SIN_DATO,SIN_DATO,Sur,Herido,ALTA,0000-00-00 00:00:00
13751,CRU-00646048-21,NaT,15,Antonio Nari¤o,30.0,A¤os,FEMENINO,Centro Oriente,Intoxicaci¢n,ALTA,0000-00-00 00:00:00
13752,CRU-00646050-21,NaT,19,Ciudad Bol¡var,16.0,A¤os,MASCULINO,Sur,Evento Respiratorio,CRITICA,30/12/2021 23:34


In [166]:
col_recep = "RECEPCION"
df_no_dupl[col_recep]

0             1/12/2021 3:40
1             1/12/2021 3:38
2             1/12/2021 4:12
3        0000-00-00 00:00:00
4             1/12/2021 3:25
                ...         
13749    0000-00-00 00:00:00
13750    0000-00-00 00:00:00
13751    0000-00-00 00:00:00
13752       30/12/2021 23:34
13753    0000-00-00 00:00:00
Name: RECEPCION, Length: 13754, dtype: object

In [167]:
# Más de limpieza de fechas o datetimes
from dateutil.parser import parse
fecha = "may 29th 2021"
#pd.to_datetime(fecha)
parse(fecha)


datetime.datetime(2021, 5, 29, 0, 0)

In [168]:
fechas_fx = lambda x: parse(x)
list_fechas = list()
for fecha in df_no_dupl[col_recep]:
    try:
        new_date = parse(fecha)
    except Exception as e:
        new_date = pd.to_datetime(fecha, errors ="coerce")
        #print(e,new_date)
    list_fechas.append(new_date)
df_no_dupl["RECEPCION_corr"]=list_fechas
df_no_dupl.head(10)

C:\Users\57320\AppData\Local\Temp\ipykernel_9768\1597762741.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_dupl["RECEPCION_corr"]=list_fechas


,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,RECEPCION,RECEPCION_corr
0,CRU-00595688-21,NaT,18,Rafael Uribe Uribe,67.0,A¤os,MASCULINO,Sur,Enfermo,MEDIA,1/12/2021 3:40,2021-01-12 03:40:00
1,CRU-00595689-21,NaT,11,Suba,NaN,SIN_DATO,SIN_DATO,Norte,Herido,BAJA,1/12/2021 3:38,2021-01-12 03:38:00
2,CRU-00595690-21,NaT,4,San Crist¢bal,34.0,A¤os,MASCULINO,Centro Oriente,Enfermo,MEDIA,1/12/2021 4:12,2021-01-12 04:12:00
3,CRU-00595693-21,NaT,7,Bosa,NaN,SIN_DATO,SIN_DATO,Sur Occidente,Trastorno Mental,CRITICA,0000-00-00 00:00:00,NaT
4,CRU-00595694-21,NaT,3,Santa Fe,61.0,A¤os,FEMENINO,Centro Oriente,Accidente Cerebro Vascular,CRITICA,1/12/2021 3:25,2021-01-12 03:25:00
5,CRU-00595699-21,NaT,19,Ciudad Bol¡var,15.0,A¤os,MASCULINO,Sur,Trastorno Mental,MEDIA,0000-00-00 00:00:00,NaT
6,CRU-00595706-21,NaT,8,Kennedy,NaN,SIN_DATO,SIN_DATO,Sur Occidente,Enfermo,BAJA,1/12/2021 4:17,2021-01-12 04:17:00
7,CRU-00595712-21,NaT,2,Chapinero,NaN,SIN_DATO,SIN_DATO,Norte,Herido,ALTA,1/12/2021 5:12,2021-01-12 05:12:00
8,CRU-00595709-21,NaT,10,Engativ,22.0,A¤os,FEMENINO,Sur Occidente,Trastorno Mental,MEDIA,1/12/2021 2:37,2021-01-12 02:37:00
9,CRU-00595710-21,NaT,7,Bosa,70.0,A¤os,MASCULINO,Sur Occidente,Dolor Tor cico,ALTA,0000-00-00 00:00:00,NaT


In [169]:
df_no_dupl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13754 entries, 0 to 13753
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   NUMERO_INCIDENTE                   13754 non-null  object        
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  0 non-null      datetime64[ns]
 2   CODIGO_LOCALIDAD                   13754 non-null  int64         
 3   LOCALIDAD                          13754 non-null  object        
 4   EDAD                               7156 non-null   float64       
 5   UNIDAD                             13754 non-null  object        
 6   GENERO                             13754 non-null  object        
 7   RED                                13754 non-null  object        
 8   TIPO_INCIDENTE                     13754 non-null  object        
 9   PRIORIDAD                          13754 non-null  object        
 10  RECEPCION                         

In [60]:
# quitar espacios en blanco al inicio o final de una palabra
word1 = "Big Data"
word2 = " BigData"
print(word1.strip(),word2.strip()) #strip es la funcion

Big Data BigData


In [20]:
word1.replace(" ","_")

'Big_Data'

In [21]:
word1.upper() #todas mayus
word2.lower() #todas minus
word1.capitalize() #solo la primera mayus

'Big data'

In [22]:
Col = "LOCALIDAD"
df_no_dupl[Col].value_counts()

Kennedy               1740
Suba                  1323
Engativï¿½            1304
Bosa                  1065
Ciudad Bolï¿½var      1009
Puente Aranda          742
Fontibï¿½n             670
Usaquï¿½n              667
Rafael Uribe Uribe     601
Usme                   597
San Cristï¿½bal        595
Chapinero              507
Tunjuelito             391
Barrios Unidos         389
Los Mï¿½rtires         379
Teusaquillo            361
Santa Fe               356
Antonio Nariï¿½o       244
La Candelaria           85
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [23]:
import re

def encontrar_patron(patron, txt):
    if re.search(patron,txt):
        new_txt = "Los Martires"
        return new_txt
    else:
        pass

encontrar_patron("Los.*tires", "Los Mï¿½rtires")

'Los Martires'

In [170]:
out_name = "reporte_compilado.csv"
out_path = os.path.join(data_source,"data","processed", out_name)
    #new_data.to_csv(out_path, mode="a", header=not os.path.isfile(out_path) and os.stat(out_path).st_size != 0) #comprobamos que es un archivo y ademas no esta vació
df_no_dupl.to_csv(out_path, mode="a")